In [2]:
import random
import csv
import sys
csv.field_size_limit(sys.maxsize)
import pandas as pd
import baseline_cands as bc
import torch

In [ ]:
# get data for baselines (Beam Search, Beam Size 1, 10, 50)
testpreds = pd.read_csv("scontcompresseddef.csv")

In [ ]:
btok, bmodel = bc.load_model("fr_en", "cuda:2")
bmodel.eval()

In [ ]:
with torch.no_grad():
    bscands = bc.get_generate_candidates(bc.beam_generate, testpreds['src'], testpreds['ref']
                , btok, bmodel, 10, 1, "fr_en", "cuda:2", "beam", 80)

In [25]:
(torch.triu(torch.ones(32, 32))*2-torch.ones(32, 32))*-1

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [ 1.,  1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1.,  1.,  1.,  ...,  1., -1., -1.],
        [ 1.,  1.,  1.,  ...,  1.,  1., -1.]])

In [10]:
sum(bscands[1])/len(bscands[1])

0.9539082969432306

In [23]:
a = torch.tensor([[10, 20,0, 30.], [10, 20, 3, 30.]])/10

In [24]:
torch.sum(a>0)

tensor(7)

In [4]:
import json

In [5]:
def convert_wmt ():
    #TODO there's definetely a way to clean this up
    with open("translation_data/news-commentary-v15.en-fr.tsv") as file:
        tsv_file = csv.reader(file, delimiter="\t")
        i = 0
        res = []
        # generate data for everything, shuffle and sample later
        for f in tsv_file:
            if i == 100000:
                break
            res.append(f)
            i = i+1
    random.shuffle(res)
    print("GOT HERE SOMEHOW")
    res = res[:10000]
    tmpdf = pd.DataFrame(res)
    tmpdf['en'] = tmpdf[0]
    tmpdf['fr'] = tmpdf[1]
    del tmpdf[0]
    del tmpdf[1]
    with open('mt-data/use/fr-en.fr', 'w') as f:
        for line in tmpdf['fr']:
            f.write(line)
            f.write('\n')
    with open('mt-data/use/fr-en.en', 'w') as f:
        for line in tmpdf['en']:
            f.write(line)
            f.write('\n')
    
    return tmpdf

In [6]:
def load_cands(fname):
    data = []
    with open(fname, 'r') as file:
        while True:
            line = file.readline()
            # if line is empty
            # end of file is reached
            if not line or len(line)<3:
                break

            data.append(json.loads(line))
    return data

In [7]:
german= load_cands("./candoutputs/explodedmt1n_en-de_bfs_recom_4_80_False_0.4_True_False_4_5_rcb_0.904_0.0_0.9.jsonl")

In [5]:
french = load_cands("./candoutputs/explodedmtn1_fr-en_bfs_recom_4_-1_False_0.4_True_False_4_5_rcb_0.904_0.0_0.9.jsonl")

In [9]:
def lset_tocominp(lset):
    res = []
    for g in lset:
        if len(g['cands'])<32:
            continue
        samp = random.sample(g['cands'], 32)+[g['ref']]
        for s in samp:
            tmp = {}
            tmp['src'] = g['inp']
            tmp['hyp'] = s
            tmp['ref'] = g['ref']
            res.append(tmp)
    return res

In [10]:
cominpde = lset_tocominp(german)

In [12]:
cpfr = pd.DataFrame(cominpde)

In [14]:
cpfr.to_csv("./processeddata/cpgdata.csv")

In [18]:
cpfr[:64]['hyp']

0     After all, as an investigative reporter in the...
1     After all, as a field investigative journalist...
2     After all, as an investigative reporter in the...
3     After all, as a journalist investigating the c...
4     After all, as a journalist investigating in th...
                            ...                        
59    After all, as a journalist investigating the c...
60    After all, as an investigative reporter in the...
61    After all, as an investigative journalist in t...
62    After all, as an investigative reporter in the...
63    After all, as an investigative journalist on t...
Name: hyp, Length: 64, dtype: object

In [4]:
cpfr = pd.read_csv("./processeddata/cpgdata1.csv")

In [2]:
import pandas as pd

In [5]:
cpfr = cpfr.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [6]:
cpfr

,src,hyp,ref,score
0,"But it is also dangerous, because it derives f...","Aber es ist auch gefährlich, da es eher von de...","Doch ist sie auch gefährlich, da sie sich aus ...",0.639357
1,"But it is also dangerous, because it derives f...","Aber es ist auch gefährlich, da es eher von de...","Doch ist sie auch gefährlich, da sie sich aus ...",0.195403
2,"But it is also dangerous, because it derives f...","Doch es ist auch gefährlich, da es eher von de...","Doch ist sie auch gefährlich, da sie sich aus ...",0.427241
3,"But it is also dangerous, because it derives f...","Doch es ist auch gefährlich, weil es sich eher...","Doch ist sie auch gefährlich, da sie sich aus ...",0.164425
4,"But it is also dangerous, because it derives f...","Aber es ist auch gefährlich, weil es sich eher...","Doch ist sie auch gefährlich, da sie sich aus ...",0.183543
...,...,...,...,...
370321,"Second, all countries have the right to the pe...",Zweitens haben alle Länder das Recht auf die f...,Zweitens haben alle Länder das Recht auf die f...,0.931637
370322,"Second, all countries have the right to the pe...",Zweitens haben alle Länder Recht auf die fried...,Zweitens haben alle Länder das Recht auf die f...,0.925203
370323,"Second, all countries have the right to the pe...",Zweitens haben alle Länder das Recht auf fried...,Zweitens haben alle Länder das Recht auf die f...,0.907322
370324,"Second, all countries have the right to the pe...",Zweitens haben alle Länder das Recht auf eine ...,Zweitens haben alle Länder das Recht auf die f...,0.802918


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

2022-10-22 10:34:01.435530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-22 10:34:01.435556: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"


In [9]:
def get_mbart_nll(inp, out, inptok, labtok, mod, dev):

    inputs = inptok(inp, return_tensors="pt").to(dev)
    with labtok.as_target_tokenizer():
        labels = labtok(out, return_tensors="pt").to(dev)

    # forward pass
    output = mod(**inputs, labels=labels.input_ids)
    #print(type(labels))
    #print(labels.attention_mask)
    return output.loss

def rescore_cands(dset, hyplist, srclist):
    device = "cuda:1" if torch.cuda.is_available() else "cpu"
    if "de" in dset:
        mname = "facebook/mbart-large-50-one-to-many-mmt"
        src_l = "en_XX"
        tgt_l = "de_DE"
    else:
        mname = "facebook/mbart-large-50-many-to-one-mmt"
        src_l = "fr_XX"
        tgt_l = "en_XX"
    inptok = AutoTokenizer.from_pretrained(mname)
    labtok = AutoTokenizer.from_pretrained(mname, src_lang=src_l, tgt_lang=tgt_l)
    mod = AutoModelForSeq2SeqLM.from_pretrained(mname)
    mod.to(device)
    print("rescoring candidates")
    i = 0
    result = []
    for i in range(0, len(hyplist)):
        if i%500==0:
            print(i)
        try:
            result.append(float(get_mbart_nll(srclist[i], hyplist[i], inptok, labtok, mod, device)))
        except:
            result.append(0)
            continue
        #print(i)
        i+=1
    del inptok
    del labtok
    del mod
    torch.cuda.empty_cache()
    return result

In [10]:
# cache dir for cometqe model
cometqe_dir = "./cometqemodel"
# can alternatively use wmt21-comet-qe-mqm
cometqe_model = "wmt20-comet-qe-da"

def get_cometqe_scores(hyps, srcs, commodel):
    cometqe_input = [{"src": src, "mt": mt} for src, mt in zip(srcs, hyps)]
    # sentence-level and corpus-level COMET
    outputs = commodel.predict(
        cometqe_input, batch_size=32, progress_bar=True
    )
    torch.cuda.empty_cache()
    return outputs

In [11]:
from comet import download_model, load_from_checkpoint

cometqe_path = download_model(cometqe_model, cometqe_dir)
model = load_from_checkpoint(cometqe_path).to(device)

wmt20-comet-qe-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [13]:
qescores = get_cometqe_scores(cpfr['hyp'], cpfr['src'], model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Predicting: 100%|████████████████████████████████████████████████████████████████████████████████████████| 11573/11573 [41:02<00:00,  4.70it/s]


In [14]:
len(qescores[0])

370326

In [15]:
cpfr['comscores'] = qescores[0]

In [16]:
del model 
del cometqe_path

In [ ]:
res = rescore_cands('fr-en', cpfr['hyp'], cpfr['src'])
cpfr['posthoc'] = res

In [18]:
cpfr.to_csv("./processeddata/cpgdata2.csv")

In [19]:
cpfr

,src,hyp,ref,score,comscores,posthoc
0,"But it is also dangerous, because it derives f...","Aber es ist auch gefährlich, da es eher von de...","Doch ist sie auch gefährlich, da sie sich aus ...",0.639357,0.622835,5.242762
1,"But it is also dangerous, because it derives f...","Aber es ist auch gefährlich, da es eher von de...","Doch ist sie auch gefährlich, da sie sich aus ...",0.195403,0.492898,4.967160
2,"But it is also dangerous, because it derives f...","Doch es ist auch gefährlich, da es eher von de...","Doch ist sie auch gefährlich, da sie sich aus ...",0.427241,0.493221,5.173780
3,"But it is also dangerous, because it derives f...","Doch es ist auch gefährlich, weil es sich eher...","Doch ist sie auch gefährlich, da sie sich aus ...",0.164425,0.159874,5.311586
4,"But it is also dangerous, because it derives f...","Aber es ist auch gefährlich, weil es sich eher...","Doch ist sie auch gefährlich, da sie sich aus ...",0.183543,0.328871,5.030740
...,...,...,...,...,...,...
370321,"Second, all countries have the right to the pe...",Zweitens haben alle Länder das Recht auf die f...,Zweitens haben alle Länder das Recht auf die f...,0.931637,0.866125,4.463050
370322,"Second, all countries have the right to the pe...",Zweitens haben alle Länder Recht auf die fried...,Zweitens haben alle Länder das Recht auf die f...,0.925203,0.873090,4.635678
370323,"Second, all countries have the right to the pe...",Zweitens haben alle Länder das Recht auf fried...,Zweitens haben alle Länder das Recht auf die f...,0.907322,0.877118,4.922931
370324,"Second, all countries have the right to the pe...",Zweitens haben alle Länder das Recht auf eine ...,Zweitens haben alle Länder das Recht auf die f...,0.802918,0.788950,5.055133
